In [ ]:
# -*- coding: utf-8 -*-
"""
Eduardo Brezolin e Vitor Carvalho
Adaptado de:https://www.jcbrolabs.org/python-codes  
Data: Abril 2022
"""
import os
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as tk
import numpy as np
import serial as sr
from tkinter.filedialog import askdirectory
# add figure canvas
fig = Figure();
figCtrl = Figure();
figErro = Figure();
#------global variables
y = np.array([])
r = np.array([])
e = np.array([]) # Sinal de erro
u = np.array([])
k = np.array([],dtype = np.int_)
P = np.array([])
I = np.array([])
D = np.array([])
Ureal = np.array([])
i = 0
ctrlArrayIndex = 0
cond = False
numAmostras = 500
#-----plot data-----
def escolhePlot():
    global ctrlArrayIndex,pltControle,lineControle,numAmostras,figCtrl,canvasCtrl
    figCtrl.clear()
    pltControle = figCtrl.add_subplot(111)
    pltControle.set_xlabel('Últimas Amostras')#'Últimas 500 Amostras')
    pltControle.set_ylabel('Tensão(V)')
    if(ctrlArrayIndex==0):
        
        pltControle.set_title('Sinal De Controle')#'Referência e Saída')
        pltControle.yaxis.set_ticks(np.arange(0.0, 10.0, 1)) 
        pltControle.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
        pltControle.set_xlim(0,numAmostras)
        pltControle.set_ylim(-0.5,10.5)
        lineControle = pltControle.plot([],[],label = 'Controle (u[k])',color ='#06069c')[0] #'Referência')[0] 
        pltControle.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
        pltControle.grid()
    elif(ctrlArrayIndex==1):
        pltControle.set_title('Sinal da Ação Proporcional')#'Referência e Saída')
        pltControle.yaxis.set_ticks(np.arange(-10.0, 10.0, 1))
        pltControle.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
        pltControle.set_xlim(0,numAmostras)
        pltControle.set_ylim(-10.5,10.5)
        lineControle = pltControle.plot([],[],label = 'P[k])',color = '#000000')[0] #'Referência')[0]
        pltControle.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
        pltControle.grid()
        
    elif(ctrlArrayIndex==2):
        pltControle.set_title('Sinal da Ação Integral')#'Referência e Saída')
        pltControle.yaxis.set_ticks(np.arange(0.0, 10.0, 1)) 
        pltControle.set_xlim(0,numAmostras)
        pltControle.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
        pltControle.set_ylim(-0.5,10.5)
        lineControle = pltControle.plot([],[],label = 'I[k])',color ='#20610d')[0] #'Referência')[0]
        pltControle.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
        pltControle.grid()
        
    elif(ctrlArrayIndex==3):
        pltControle.set_title('Sinal da Ação Derivativa')#'Referência e Saída')
        pltControle.yaxis.set_ticks(np.arange(-1, 1, 0.25)) 
        pltControle.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
        pltControle.set_xlim(0,numAmostras)
        pltControle.set_ylim(-1.05,1.05)
        lineControle = pltControle.plot([],[],label = 'D[k])',color = '#8e27c2')[0] #'Referência')[0]
        pltControle.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
        pltControle.grid()
    elif(ctrlArrayIndex==4):
        pltControle.set_title('Sinal de Controle Ideal')#'Referência e Saída')
        pltControle.yaxis.set_ticks(np.arange(-18.0, 18.0, 3))  
        pltControle.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
        pltControle.set_xlim(0,numAmostras)
        pltControle.set_ylim(-18,18)
        lineControle = pltControle.plot([],[],label = 'Uideal[k])',color ='#0000ff')[0] #'Referência')[0]
        pltControle.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
        pltControle.grid()
    canvasCtrl.draw()

def next_plot():
    global ctrlArrayIndex
    ctrlArrayIndex = ctrlArrayIndex +1
    print(ctrlArrayIndex)
    if (ctrlArrayIndex==5):
        ctrlArrayIndex =0
        
    escolhePlot()
    
def prev_plot():
    global ctrlArrayIndex
    ctrlArrayIndex =ctrlArrayIndex -1
    if (ctrlArrayIndex==-1):
        ctrlArrayIndex =4
    escolhePlot()
    
def plot_data():
    global cond, numAmostras,i,r,y,u,e,k, P,I,D,Ureal,ctrlArrayIndex
    #FORMATO REFERENCIA # SAIDA # CONTROLE # ERRO
    if (cond == True):
        # faz a "decodificação do protolo dado# dado# dado\r\n"
        a = s.readline()
        a = a.decode()
        str = a[:-2] # joga fora o \r e o \n
        str = str.split('#')
        i+=1
        if(len(r) < numAmostras):
            r = np.append(r,float(str[0])) # adicionar elementos conforme a necessidade
            y = np.append(y,float(str[1]))
            u = np.append(u,float(str[2]))
            e = np.append(e,float(str[3]))
            P = np.append(P,float(str[4]))
            I = np.append(I,float(str[5]))
            D = np.append(D,float(str[6]))
            Ureal = np.append(Ureal,float(str[7]))
            k = np.append(k,i) 
        else:#extremamamente hard coded 
            r[0:numAmostras-1] = r[1:numAmostras]
            r[numAmostras-1]   = float(str[0])
            y[0:numAmostras-1] = y[1:numAmostras]
            y[numAmostras-1]   = float(str[1])
            u[0:numAmostras-1] = u[1:numAmostras]
            u[numAmostras-1]   = float(str[2])
            e[0:numAmostras-1] = e[1:numAmostras]
            e[numAmostras-1]   = float(str[3])
            P[0:numAmostras-1] = P[1:numAmostras]
            P[numAmostras-1]   = float(str[4])
            I[0:numAmostras-1] = I[1:numAmostras]
            I[numAmostras-1]   = float(str[5])
            D[0:numAmostras-1] = D[1:numAmostras]
            D[numAmostras-1]   = float(str[6])
            Ureal[0:numAmostras-1] = Ureal[1:numAmostras]
            Ureal[numAmostras-1]   = float(str[7])
            k[0:numAmostras-1] = k[1:numAmostras]
            k[numAmostras-1]   = i
            
            
        lineReferencia.set_xdata(np.arange(0,len(r)))
        lineReferencia.set_ydata(r)
        linePlanta.set_xdata(np.arange(0,len(y)))
        linePlanta.set_ydata(y)
        canvasMain.draw()
        
        
        if (ctrlArrayIndex ==0):
            lineControle.set_xdata(np.arange(0,len(u)))
            lineControle.set_ydata(u)
        elif (ctrlArrayIndex ==1):
            lineControle.set_xdata(np.arange(0,len(P)))
            lineControle.set_ydata(P)
        elif (ctrlArrayIndex ==2):
            lineControle.set_xdata(np.arange(0,len(I)))
            lineControle.set_ydata(I)
        elif (ctrlArrayIndex ==3):
            lineControle.set_xdata(np.arange(0,len(D)))
            lineControle.set_ydata(D)
        elif (ctrlArrayIndex ==4):
            lineControle.set_xdata(np.arange(0,len(Ureal)))
            lineControle.set_ydata(Ureal)
        
        canvasCtrl.draw()
        
        lineErro.set_xdata(np.arange(0,len(e)))
        lineErro.set_ydata(e)
        canvasErro.draw()
    
    root.after(1,plot_data)

def on_closing():
    global s
    s.close()
    root.destroy()

def save_data():
    global r,y,u,e,k, P,I,D,Ureal
    plot_stop();
    img_dir = askdirectory()
    os.chdir(img_dir)
    salvar = np.column_stack((k,r,y,u,e,P,I,D,Ureal))
    np.savetxt("dados.csv", salvar,fmt='%5d , %.4e , %.4e, %.4e, %.4e, %.4e , %.4e, %.4e, %.4e' )
    
def plot_start():
    global cond
    cond = True
    s.reset_input_buffer()

def plot_stop():
    global cond
    cond = False
def save_fig():
    global fig,figCtrl,figErro
    plot_stop();
    img_dir = askdirectory()
    os.chdir(img_dir)
    fig.savefig('MF.png')
    figCtrl.savefig('CTRL.png')
    figErro.savefig('ERRO.png')
    
def reset_data():
    global cond,r,y,u,e,k, P,I,D,Ureal
    plot_stop()
    i=0;
    y = np.array([])
    r = np.array([])
    e = np.array([]) # Sinal de erro
    u = np.array([])
    k = np.array([],dtype = np.int_)
    P = np.array([])
    I = np.array([])
    D = np.array([])
    Ureal = np.array([])
    
    lineReferencia.set_xdata(np.arange(0,len(r)))
    lineReferencia.set_ydata(r)
    linePlanta.set_xdata(np.arange(0,len(y)))
    linePlanta.set_ydata(y)
    canvasMain.draw()
    
    lineControle.set_xdata(np.arange(0,len(u)))
    lineControle.set_ydata(u)
    canvasCtrl.draw()
    
    lineErro.set_xdata(np.arange(0,len(e)))
    lineErro.set_ydata(e)
    canvasErro.draw()



  
    
    
#-----Main GUI code-----
root = tk.Tk()
root.title('CONTROLE DIGITAL - EDUARDO E VITOR')
root.configure(background = 'light blue')
#root.geometry("525X500") # set the window size
# Modificações 
root.state('zoomed') # full screen


#------create Plot object on GUI----------
pltErro = figErro.add_subplot(111)
pltControle = figCtrl.add_subplot(111)
pltMain = fig.add_subplot(111)

# criação das figuras INDIVIDUAS 

# PLOT PRINCIPAL
pltMain.set_title('Malha Fechada')#'Referência e Saída')
pltMain.set_xlabel('Amostras')#'Últimas 500 Amostras')
pltMain.set_ylabel('Tensão(V)')
pltMain.yaxis.set_ticks(np.arange(0.0, 10.0, 1))
pltMain.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
pltMain.set_xlim(0,numAmostras)
pltMain.set_ylim(-0.5,10.5)
lineReferencia = pltMain.plot([],[],label = 'Entrada (r[k])')[0] #'Referência')[0]
linePlanta = pltMain.plot([],[],label = 'Saída (y[k])')[0]#'Planta')[0] # adicionar linhas com o que quero
pltMain.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
pltMain.grid()
canvasMain = FigureCanvasTkAgg(fig, master=root)  # A tk.DrawingArea.
canvasMain.get_tk_widget().place(x = 15,y=15, width = 895, height = 765)
canvasMain.draw()


pltControle.set_xlabel('Últimas Amostras')#'Últimas 500 Amostras')
pltControle.set_ylabel('Tensão(V)')
pltControle.set_xlim(0,numAmostras)
    #if (ctrlArrayIndex =0)
pltControle.set_title('Sinal De Controle')#'Referência e Saída')
pltControle.yaxis.set_ticks(np.arange(0.0, 10.0, 1)) 
pltControle.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
pltControle.set_ylim(-0.5,10.5)
lineControle = pltControle.plot([],[],label = 'Controle (u[k])',color ='#06069c')[0] #'Referência')[0]
    #plantaMain = pltMain.plot([],[],label = 'Saída')[0]#'Planta')[0] # adicionar linhas com o que quero
pltControle.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
pltControle.grid()
canvasCtrl = FigureCanvasTkAgg(figCtrl, master=root)  # A tk.DrawingArea.
canvasCtrl.get_tk_widget().place(x = 920, y=15, width = 600,height = 400)
canvasCtrl.draw()

# PLOT ERRO

pltErro.set_title('Erro percetual')#'Referência e Saída')
pltErro.set_xlabel('Amostras')#'Últimas 500 Amostras')
pltErro.set_ylabel('Tensão(V)')
pltErro.yaxis.set_ticks(np.arange(-15.0, 15.0, 5))
pltErro.xaxis.set_ticks(np.arange(0.0, numAmostras, numAmostras/10)) 
pltErro.set_xlim(0,numAmostras)
pltErro.set_ylim(-15.5,15.5)
lineErro = pltErro.plot([],[],label = 'Erro %(e%[k])',color = '#db0909')[0]#'Planta')[0] # adicionar linhas com o que quero
pltErro.legend(fancybox=True, framealpha=0.5,  borderpad=1,loc ='upper right')
pltErro.grid()

canvasErro = FigureCanvasTkAgg(figErro, master=root)  # A tk.DrawingArea.
canvasErro.get_tk_widget().place(x = 920, y=425, width = 600,height = 400)
canvasErro.draw()
    
#----------create button---------
root.update();
start = tk.Button(root, text = "Iniciar", font = ('calbiri',12),command = lambda: plot_start())
start.place(x = 200, y = 795 )

root.update();
stop = tk.Button(root, text = "Parar", font = ('calbiri',12), command = lambda:plot_stop())
stop.place(x = 275, y = 795)

root.update();
svfig = tk.Button(root, text = "Salvar Imagem", font = ('calbiri',12), command = lambda:save_fig())
svfig.place(x = 350, y =795)

root.update();
svdata = tk.Button(root, text = "Salvar Dados", font = ('calbiri',12), command = lambda:save_data())
svdata.place(x = 480, y = 795)

root.update();
rst = tk.Button(root, text = "Reset", font = ('calbiri',12), command = lambda:reset_data())
rst.place(x = 625, y = 795)

root.update();
rst = tk.Button(root, text = "=>", font = ('calbiri',12), command = lambda:next_plot())
rst.place(x = 1475, y = 25)
root.update();
rst = tk.Button(root, text = "<=", font = ('calbiri',12), command = lambda:prev_plot())
rst.place(x = 935, y = 25)


#----start serial port----
s = sr.Serial('COM7',9600);
s.reset_input_buffer()

root.resizable(False, False)
root.protocol("WM_DELETE_WINDOW", on_closing)
root.after(1,plot_data)
root.mainloop()

5
1
2
4
